In [1]:
import torch 
import cv2
import numpy as np
import os 
import glob
from torch.utils.data import DataLoader,Dataset
from torch import optim,nn 
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import torchvision


In [2]:

writer = SummaryWriter("objectLoc/runs")

2022-11-24 16:15:19.724852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 16:15:19.856258: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-24 16:15:19.859860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/local/ZOHOCORP/kala-pt5650/Python-Files/Python3.10.4/bin/venv3.9/lib/python3.9/site-packag

In [3]:
np.random.seed(1)
torch.manual_seed(1)
class ShapeDataset(Dataset):
    def __init__(self,path):
        self.class_label = {"circle":0,"rectangle":1,"triangle":2,"square":3,"star":4,"bg":5}
        self.Image_path_list = []

        for data_path in glob.glob(path+"**/*.jpg"):
            self.Image_path_list.append(data_path)
       
        np.random.shuffle(self.Image_path_list)
        
    def get_label_box(self,csvFileName,  image_size ):
       
        df = pd.read_csv(csvFileName,
        usecols=['ClassName','X','Y','Width','Height'])
        df = np.array(df)[0]
        # print(df[1:].astype(np.float32))
        label ,boxes = self.class_label[df[0]],(df[1:].astype(np.float32)/image_size)
        boxes = torch.from_numpy(boxes)
        return label,boxes

    def __getitem__(self, index):
        img = cv2.imread(self.Image_path_list[index],0)
        image_size = 224
        img = cv2.resize(img,(image_size,  image_size ),interpolation=cv2.INTER_AREA).reshape(1,image_size,image_size)
        img = torch.from_numpy(img)/255.0
        one_hot_label = torch.zeros(6,dtype=torch.float32)
        label,boxes = self.get_label_box(self.Image_path_list[index][:-4]+".csv",  image_size )
        one_hot_label[label] = 1
        return img,boxes,one_hot_label
    def __len__(self):
        return len(self.Image_path_list)

In [4]:

""" Dataset initalization  """

train_path = "ShapeData/Train"
test_path = "ShapeData/Test"
train_dataset = ShapeDataset(train_path)

test_dataset = ShapeDataset(test_path)
print("train :",len(train_dataset),"test:",len(test_dataset),"total:",len(train_dataset)+len(test_dataset))
# plt.imshow(tarin_dataset[0][0].T)
train_dataset[0][-1]

train : 2500 test: 500 total: 3000


tensor([0., 0., 0., 1., 0., 0.])

In [5]:
class ConvBn(nn.Module):
    def __init__(self,input_ch,out_ch,stride):
        super().__init__()
        self.Conv = nn.Sequential(

            nn.Conv2d(in_channels=input_ch,out_channels=out_ch,kernel_size=3,stride=stride,padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )

    def forward(self,x):
        x = self.Conv(x)
        return x 

In [6]:
class DepthWiseSeparableConv(nn.Module):
    def __init__(self,input_ch,out_ch,stride=1):
        super().__init__()
        # Depthwise convolution
        self.ConvDw = nn.Sequential(
        nn.Conv2d(in_channels=input_ch,out_channels=input_ch,kernel_size=3,stride=stride,padding=1,groups=input_ch),
        nn.BatchNorm2d(num_features=input_ch),
        nn.ReLU()
        )
        # Pointwise convolution
        self.ConvPw = nn.Sequential(

            nn.Conv2d(in_channels=input_ch,out_channels=out_ch,kernel_size=1,stride=1),
            nn.BatchNorm2d(num_features=out_ch),
            nn.ReLU()
        )
    def forward(self,x):
        x = self.ConvDw(x)
        x = self.ConvPw(x)
        return x 


In [7]:
class MobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(ConvBn(1,32,1),# 224
                    DepthWiseSeparableConv(32,64,2),# 112
                    DepthWiseSeparableConv(64,128,2),# 56
                    DepthWiseSeparableConv(128,128,1),# 56
                    DepthWiseSeparableConv(128,256,2),# 28
                    DepthWiseSeparableConv(256,256,1), # 28
                    DepthWiseSeparableConv(256,512,2), # 14
                    DepthWiseSeparableConv(512,512,1),# 14
                    DepthWiseSeparableConv(512,256,2),# 7
                    nn.AvgPool2d(7))

       
        self.fc = nn.Linear(256,256)
        self.class_fc = nn.Linear(256,6)
        self.box_fc = nn.Linear(256,4)

    def forward(self,x):
        x = self.layers(x)
        x = x.view(-1,256)
        x = self.fc(x)
        class_x = self.class_fc(x)
        box_x = self.box_fc(x)
        return class_x,box_x

In [8]:

batch_size = 8
drop_last = True
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=4, drop_last=drop_last)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True,num_workers=4, drop_last=drop_last)

In [9]:
class_loss_func = nn.CrossEntropyLoss()
boxes_loss_func = nn.MSELoss()
# boxes_loss_func2 = nn.MSELoss()
model = MobileNet()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [10]:
images,boxes,labels= next(iter(train_loader))
img_grid = torchvision.utils.make_grid(images)
writer.add_image('shape_images', img_grid)
writer.add_graph(model,images)

In [11]:
# boxes_loss_func1 = nn.MSELoss(reduce=False)
# boxes_loss_func2 = nn.MSELoss()
# x1 = torch.tensor([[0.89,0.67,0.45,0.56],[0.89,0.67,0.45,0.56]])
# x2 = torch.tensor([[0.89,0.56,0.67,0.89],[0.89,0.56,0.67,0.89]])
# # x2 = torch.nan
# l1 = boxes_loss_func1(x1,x2)
# l2 = boxes_loss_func2(x1,x2)
# l1,l2,torch.mean((x1-x2)**2)
# # x1*x2


In [12]:
print_format = 'Epoch {:03}: | Train Loss: {:.5f} | Test Loss:{:.5f} | val_accuracy: {:.5f}'

total_batches_for_train = len(train_dataset)/batch_size
total_batches_for_train = np.floor(total_batches_for_train) if drop_last else np.ceil(total_batches_for_train)

total_batches_for_test = len(test_dataset)/batch_size
total_batches_for_test = np.floor(total_batches_for_test) if drop_last else np.ceil(total_batches_for_test)

print(total_batches_for_train)

312.0


In [13]:
# for x_train , b_train , y_train in train_loader :
#     print(x_train.shape,b_train.shape,y_train.shape)


In [14]:
def train(model,train_loader,test_dataset,class_loss_func,boxes_loss_func,epochs=100):

    for epoch in range(epochs):
        total_train_loss,total_test_loss,correct=0,0,0

        model.train()
        
        for x_train , b_train , y_train in train_loader :
            optimizer.zero_grad()
            z_train = model(x_train)
            # class loss
            train_class_loss = class_loss_func(z_train[0],y_train)
            # box loss 
            nan = torch.isnan(b_train)
            B_train = torch.where(nan, torch.tensor(0.0), b_train)
            BZ_train  = torch.where(nan, torch.tensor(0.0), z_train[1])
            train_box_loss = boxes_loss_func(BZ_train,B_train)

            # total train loss
            train_loss = train_box_loss + train_class_loss
           
            #backprop
            train_loss.backward()
            optimizer.step()
            
            total_train_loss +=train_loss.item()
            
        model.eval()
        for x_test, b_test , y_test in test_loader:
            z_test = model(x_test)
            test_class_loss = class_loss_func(z_test[0],y_test)

            nan = torch.isnan(b_test)
            B_test = torch.where(nan, torch.tensor(0.0), b_test)
            # BZ_test = torch.where(nan, torch.tensor(0.0), z_test[1])
            test_box_loss = boxes_loss_func(z_test[1],B_test)

            test_loss = test_class_loss + test_box_loss 

            _,z_hat = torch.max(z_test[0],1)
            _,y_hat = torch.max(y_test,1)
            correct += (z_hat == y_hat).sum().item()
            
            total_test_loss += test_loss.item()

        total_test_loss = total_test_loss/total_batches_for_test
        total_train_loss = total_train_loss/total_batches_for_train
        val_accuracy = (correct / len(test_dataset)) * 100
        total_train_loss = total_train_loss/total_batches_for_test

        writer.add_scalar(f"Train/Train-loss",total_train_loss,epoch)
        writer.add_scalar(f"Acc/Accuracy",val_accuracy,epoch)
        writer.add_scalar(f"Train/Test-loss",total_test_loss,epoch)

        print(print_format.format(epoch,total_train_loss,total_test_loss,val_accuracy),end="\n")
        return model

In [15]:
trained_model = train(model,train_loader,test_dataset,class_loss_func,boxes_loss_func)
torch.save(trained_model.state_dict(),"ShapeModels/objloc_weights.pth")

Epoch 000: | Train Loss: 0.011 | Test Loss:0.428 | val_accuracy: 83.00000
Epoch 001: | Train Loss: 0.003 | Test Loss:3.843 | val_accuracy: 28.40000
Epoch 002: | Train Loss: 0.001 | Test Loss:0.321 | val_accuracy: 86.40000
Epoch 003: | Train Loss: 0.002 | Test Loss:0.071 | val_accuracy: 97.20000
Epoch 004: | Train Loss: 0.001 | Test Loss:0.094 | val_accuracy: 97.00000
Epoch 005: | Train Loss: 0.001 | Test Loss:0.054 | val_accuracy: 98.20000
Epoch 006: | Train Loss: 0.001 | Test Loss:0.146 | val_accuracy: 94.60000
Epoch 007: | Train Loss: 0.001 | Test Loss:0.054 | val_accuracy: 97.40000
Epoch 008: | Train Loss: 0.000 | Test Loss:0.073 | val_accuracy: 97.20000
Epoch 009: | Train Loss: 0.001 | Test Loss:0.063 | val_accuracy: 97.80000
Epoch 010: | Train Loss: 0.001 | Test Loss:0.018 | val_accuracy: 99.00000
Epoch 011: | Train Loss: 0.000 | Test Loss:0.047 | val_accuracy: 98.00000
Epoch 012: | Train Loss: 0.000 | Test Loss:0.088 | val_accuracy: 96.60000
Epoch 013: | Train Loss: 0.001 | Test 

AttributeError: 'NoneType' object has no attribute 'state_dict'